# IFT-3700 Travail 1

Félix Adam

Mehdi Aqdim

Maxime Daigle

Jonathan Graveline

## Préparation

In [3]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

"""
    Mettre 'skiprows=1' pour avoir toutes les données.
    (Mais c'est très long surtout pour faire des tests.)
"""
mnist_train = np.loadtxt('mnist_train.csv', dtype='int', delimiter=',', skiprows=59001)
mnist_test  = np.loadtxt('mnist_test.csv', dtype='int', delimiter=',', skiprows=9501)

train_label = mnist_train[:,0]
train_data  = mnist_train[:,1:]

test_label  = mnist_test[:,0]
test_data   = mnist_test[:,1:]

## Mesures

In [4]:
# distance euclidien
from sklearn.metrics import euclidean_distances

#Procrustes analysis
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.procrustes.html
from scipy.spatial import procrustes

def procrustes_similarity(u,v):
    #u and v are vectors
    # return disparity score between u and v
    u = u.reshape(u.shape[0],1)
    v = v.reshape(v.shape[0],1)
    return procrustes(u,v)[2]

def procrustes_matrix(X):
    # X is a matrix nxd with n vectors
    # return distance matrix nxn where distance [i,j] is the distance between X[i] and X[j]
    #environ 80 secondes pour creer une matrice (1000,1000)
    result = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(i, X.shape[0]):
            result[i,j] = result[j,i] = procrustes_similarity(X[i],X[j])
    return result


# Notre mesure inspiré de procrustes
# https://github.com/scipy/scipy/blob/v1.1.0/scipy/spatial/_procrustes.py#L17-L132
from scipy.linalg.decomp_svd import svd

def procrustes_modified(data1, data2):
    mtx1 = np.array(data1, dtype=np.double, copy=True)
    mtx2 = np.array(data2, dtype=np.double, copy=True)

    if mtx1.ndim != 2 or mtx2.ndim != 2:
        raise ValueError("Input matrices must be two-dimensional")
    if mtx1.shape != mtx2.shape:
        raise ValueError("Input matrices must be of same shape")
    if mtx1.size == 0:
        raise ValueError("Input matrices must be >0 rows and >0 cols")

    norm1 = np.linalg.norm(mtx1)
    norm2 = np.linalg.norm(mtx2)

    if norm1 == 0 or norm2 == 0:
        raise ValueError("Input matrices must contain >1 unique points")

    # change scaling of data (in rows) such that trace(mtx*mtx') = 1
    mtx1 /= norm1
    mtx2 /= norm2

    # measure the dissimilarity between the two datasets
    disparity = np.sum(np.square(mtx1 - mtx2))

    return mtx1, mtx2, disparity

def procrustes_similarity_modified(u,v):
    #u and v are vectors
    # return disparity score between u and v
    u = u.reshape(u.shape[0],1)
    v = v.reshape(v.shape[0],1)
    return procrustes_modified(u,v)[2]

def procrustes_matrix_modified(X):
    # X is a matrix nxd with n vectors
    # return distance matrix nxn where distance [i,j] is the distance between X[i] and X[j]
    #environ 80 secondes pour creer une matrice (1000,1000)
    result = np.zeros((X.shape[0],X.shape[0]))
    for i in range(X.shape[0]):
        for j in range(i, X.shape[0]):
            result[i,j] = result[j,i] = procrustes_similarity_modified(X[i],X[j])
    return result

## Implémentation des algorithmes

#### KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier as knn

# exemple d'utilisation 
# knn_default = knn(n_neighbors=7, metric='euclidean', n_jobs=6)
# knn_default.fit(train_data, train_label)
# result = knn_default.score(test_data, test_label)
# print("Taux de succès avec knn et la distance euclidienne: %.4f %%" % (result * 100.0))

#### Partition binaire

#### K-medoids

#### PCoA

#### Isomap

#### Visualisation de la réduction de dimensionalité

In [ ]:
def visualize_2d(pos,title):
    fig, ax = plt.subplots()
    colors=((1,0,0),(0,1,0),(0,0,1),(0.5,0.5,0),(0,0.5,0.5),(0.5,0,0.5),(0.4,0.6,0),(0.6,0.4,0),(0,0.6,0.4),(0.5,0.3,0.2),)
    for i in range(pos.shape[0]):
        ax.scatter(pos[i,0],pos[i,1],label = train_label[i],color=colors[train_label[i]])

    #remove duplicate in legend
    handles,labels=ax.get_legend_handles_labels()
    handles_unique, labels_unique = [],[]
    seen = set()
    for i,e in enumerate(labels):
        if e not in seen:
            seen.add(e)
            handles_unique.append(handles[i])
            labels_unique.append(e)
            if len(seen) == 10:
                break

    #sort legend
    labels_unique, handles_unique = (list(l) for l in zip(*sorted(zip(labels_unique, handles_unique))))

    plt.legend(handles_unique,labels_unique,loc="best")
    plt.title(title)
    plt.show()

## Présentation de la mesure choisie

Lors de la phase d'exploration, nous avons essayé de nombreuses mesures avec des résultats très variables. Par exemple, nous avons essayé la structural similarity index (SSIM)

In [9]:
from skimage.measure import compare_ssim as ssim
# http://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.compare_ssim

# mean structural similarity index between two images
start = time.time()
knn_ssim = knn(n_neighbors=7, metric=ssim, n_jobs=-1)
knn_ssim.fit(train_data, train_label)
result_ssim = knn_ssim.score(test_data, test_label)

end = time.time()
print("Taux de succès avec knn et structural similarity index: %.4f %%" % (result_ssim * 100.0))
print(end-start, 'seconds')

# Avec 5000 train example et 500 test example:
# Taux de succès avec knn et structural similarity index: 0.8000 %
# 1294.130733013153 seconds (22 minutes)

Taux de succès avec knn et structural similarity index: 0.8000 %
1294.130733013153 seconds


qui est très lent et qui obtient un résultat extrêmement mauvais. Comparativement à des conditions exactement pareilles excepté que c'est la distance euclidienne, la ssim prend environ 22 minutes sur 5000 exemples tandis que la version avec la distance euclidienne prend 0.24 secondes. De plus, la performance est de seulement 0.8% tandis que la distance euclidienne obtient 84.6%.

Nous avons aussi essayé de nombreuse mesure plus simple comme la distance cosine ou L3:

In [78]:
start = time.time()
knn_l3 = knn(n_neighbors=7, metric='minkowski', p=3, n_jobs=-1)
knn_l3.fit(train_data, train_label)
result_l3 = knn_l3.score(test_data, test_label)

end = time.time()
print("Taux de succès avec knn et L3: %.4f %%" % (result_l3 * 100.0))
print(end-start, 'seconds')

# Avec 10000 train examples et 1000 test examples:
# Taux de succès avec knn et L3: 95.5000 %
# 28.261430978775024 seconds

# Avec 5000 train examples et 500 test examples:
# Taux de succès avec knn et L3: 90.8000 %
# 6.961097002029419 seconds

# Avec 1000 train examples et 500 test examples:
# Taux de succès avec knn et L3: 85.4000 %
# 1.2310302257537842 seconds

Taux de succès avec knn et L3: 85.4000 %
1.2310302257537842 seconds


Cependant, malgré que les temps d'execution ne soient pas beaucoup plus long (ici environ 1 secondes de plus que l'euclidien), les résultats obtenus n'étaient pas très intéressant étant donné que le résultat n'étaient qu'une très faible amélioration (dans ce cas-ci une amélioration de 0.8% (de 84.6% à 85.4%)).

Finalement, la mesure trouvée qui nous a donné les résultats les plus intéressants est l'analyse procustéenne (Procrustes analysis, en anglais).

In [76]:
start = time.time()
knn_procrustes = knn(n_neighbors=7, metric=procrustes_similarity, n_jobs=-1)
knn_procrustes.fit(train_data, train_label)
result_procrustes = knn_procrustes.score(test_data, test_label)

end = time.time()
print("Taux de succès avec knn et Procrustes analysis comme similarité: %.4f %%" % (result_procrustes * 100.0))
print(end-start, 'seconds')

# Avec 10000 train examples et 1000 test examples:
# Taux de succès avec knn et Procrustes analysis comme similarité: 96.5000 %
# 2965.029656648636 seconds (49 minutes)

# Avec 5000 train example et 500 test example:
# Taux de succès avec knn et Procrustes analysis comme similarité: 92.6000 %
# 741.8232653141022 seconds (12 minutes)

# Avec 1000 train examples et 500 test examples:
# Taux de succès avec knn et Procrustes analysis comme similarité: 89.4000 %
# 126.29976058006287 seconds (2 minutes 6 secondes)

Taux de succès avec knn et Procrustes analysis comme similarité: 89.4000 %
126.29976058006287 seconds


Malgré que l'execution prend, dans cet exemple, 2 minutes 6 secondes tandis que la version euclidéenne prend seulement 0.24 secondes, nous obtenons une amélioration de presque 5% (de 84.6% à 89.4%).